In [1]:
import tensorflow as tf
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import r2_score
from tensorflow.keras import utils,losses,layers,Sequential
from tensorflow.keras.callbacks import ModelCheckpoint,TensorBoard,EarlyStopping
from tensorflow.keras.layers import Input, LSTM, Dense, Bidirectional, Dot, Softmax, Concatenate,Conv1D,Dropout,MaxPooling1D, Flatten
from tensorflow.keras.models import Model

In [2]:
data_path="winter.csv"
dataset=pd.read_csv(data_path,parse_dates=['Date'],index_col=['Date'])

future=['NH3']

for col in future:
    scaler=MinMaxScaler()
    if(col not in ['sin(h)','cos(h)']):
    	dataset[col]=scaler.fit_transform(dataset[col].values.reshape(-1,1))
dataset.head()

,Indoor_T,Indoor_RH,Outdoor_T,Outdoor_RH,NH3,TSP,CO2,TVOC,PM10,PM2.5,Wind_force,Wind_speed
Date,,,,,,,,,,,,
2023-02-08 17:20:00,18.133367,0.735792,11.923535,0.796850,0.672727,7,822,101,196,167,0,0.0
2023-02-08 17:30:00,18.208442,0.743420,11.784109,0.804846,0.727273,6,816,127,186,151,90,1.2
2023-02-08 17:40:00,18.079741,0.759696,11.698308,0.815845,0.672727,7,818,158,189,158,90,1.0
2023-02-08 17:50:00,18.283518,0.778658,11.591057,0.821781,0.600000,1335,884,1082,207,165,0,0.0
2023-02-08 18:00:00,17.586387,0.921251,11.483806,0.829049,0.636364,824,1070,2503,182,158,180,1.3


In [3]:
def multivariate_data(x,y, start_index, end_index, history_size,
                      target_size, step, single_step):
    data = []
    labels = []

    start_index = start_index + history_size
    
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i, step) 
        mid_data=x.iloc[indices]
        data.append(mid_data)

        if single_step:
            mid_data=y.iloc[i+target_size]
            labels.append(mid_data)
        else:
            labels.append(y.iloc[i:i+target_size])

    return np.array(data), np.array(labels)
x=dataset[future]
y=dataset['Indoor_RH']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,shuffle=False,random_state=25)
train_dataset,train_labels=multivariate_data(x_train,y_train,0,2800,3,1,1,False)
test_dataset,test_labels=multivariate_data(x_test,y_test,0,1200,3,1,1,False)

In [4]:
import xgboost as xgb


train_data = xgb.DMatrix(train_dataset.reshape(train_dataset.shape[0], -1), label=train_labels)
test_data = xgb.DMatrix(test_dataset.reshape(test_dataset.shape[0], -1), label=test_labels)


param = {
    'max_depth': 3,
    'eta': 0.1,
    'objective': 'reg:squarederror',
    'eval_metric': 'mae'
}


num_round = 100
model = xgb.train(param, train_data, num_round)




In [ ]:

test_preds = model.predict(test_data)

R2=r2_score(test_labels,test_preds)
print('r2:',R2)
MSE = metrics.mean_squared_error(test_labels,test_preds)
print('MSE:',MSE)
RMSE=np.sqrt(metrics.mean_squared_error(test_labels,test_preds))
print('RMSE:',RMSE)
MAE=metrics.mean_absolute_error(test_labels,test_preds)
print('MAE:',MAE)
def mape(test_labels,test_preds):
    return np.mean(np.abs((test_preds-test_labels)/test_preds))              
MAPE=mape(test_labels,test_preds)
print('MAPE:',MAPE)


In [ ]:

train_predictions = model.predict(train_data)
test_predictions = model.predict(test_data)

train_labels = train_labels.reshape(-1, 1)
train_predictions = train_predictions.reshape(-1, 1)
test_labels = test_labels.reshape(-1, 1)
test_predictions = test_predictions.reshape(-1, 1)

train_predictions = scaler.inverse_transform(train_predictions)
train_labels = scaler.inverse_transform(train_labels)
test_predictions = scaler.inverse_transform(test_predictions)
test_labels = scaler.inverse_transform(test_labels)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
train_mse = mean_squared_error(train_labels, train_predictions)
train_rmse=np.sqrt(metrics.mean_squared_error(train_labels, train_predictions))
train_mae = mean_absolute_error(train_labels, train_predictions)
train_r2 = r2_score(train_labels, train_predictions)
def train_mape(train_labels, train_predictions):
    return np.mean(np.abs((train_labels, train_predictions)/train_predictions))              
train_mape =train_mape(train_labels, train_predictions)
print('train_mse:',train_mse)
print('train_rmse:',train_rmse)
print('train_mae:',train_mae)
print('train_mape:',train_mape)
print('train_r2:',train_r2)
test_mse = mean_squared_error(test_labels, test_predictions)
test_rmse=np.sqrt(metrics.mean_squared_error(test_labels, test_predictions))
test_mae = mean_absolute_error(test_labels, test_predictions)
test_r2 = r2_score(test_labels, test_predictions)
def test_mape(test_labels, test_predictions):
    return np.mean(np.abs((test_labels, test_predictions)/test_predictions))              
test_mape =test_mape(test_labels, test_predictions)
print('test_mse:',test_mse)
print('test_rmse:',test_rmse)
print('test_mae:',test_mae)
print('test_mape:',test_mape)
print('test_r2:',test_r2)
